In [1]:
import json
import copy
import numpy as np
import pandas as pd

In [2]:
from tensorflow import keras

In [3]:
# Load yelp_academic_dataset_review_00 
review_dataset = {}
with open('data/yelp_academic_dataset_review_00',encoding='utf-8') as f:
	for line in f:
		data = json.loads(line)
		review_dataset[data['business_id']] = data


In [4]:
print(len(review_dataset))

4299


In [5]:
# Load yelp_academic_dataset_business_00
business_dataset = {}
with open('data/yelp_academic_dataset_business_00') as f:
	for line in f:
		data = json.loads(line)
		business_dataset[data['business_id']] = data

In [6]:
# Find the match between review and business_dataset
# Copy review_dataset to merged_dataset
merged_dataset = {}
for review in review_dataset:
	if review in business_dataset:
# 		print(review)
# 		print(review_dataset[review])
# 		print(business_dataset[review])
# 		print()
		# Merge the two datasets
		if review not in merged_dataset:
			merged_dataset[review] = [copy.deepcopy(review_dataset[review])]
			merged_dataset[review].append(copy.deepcopy(business_dataset[review]))
		else: 
			merged_dataset[review].append(copy.deepcopy(review_dataset[review]))
			merged_dataset[review].append(copy.deepcopy(business_dataset[review]))

In [7]:
for i in merged_dataset:
	print(merged_dataset[i])
	break

[{'review_id': 'C1uQNP2ehBktS43ZRMEvkg', 'user_id': '2M6KFsWIUXElqcQRz4A0Qg', 'business_id': 'buF9druCkbuXLX526sGELQ', 'stars': 5.0, 'useful': 1, 'funny': 2, 'cool': 0, 'text': 'Great food and service! Again. 4 out of the last 5 weeks great every time.. tuesday and Wednesday get right in.  Always looking to make sure that you have a great time!', 'date': '2014-09-10 00:06:00'}, {'business_id': 'buF9druCkbuXLX526sGELQ', 'name': 'Prides Osteria', 'address': '240 Rantoul St', 'city': 'Beverly', 'state': 'MA', 'postal_code': '01915', 'latitude': 42.5496089, 'longitude': -70.884046, 'stars': 3.5, 'review_count': 83, 'is_open': 0, 'attributes': {'OutdoorSeating': 'False', 'RestaurantsGoodForGroups': 'True', 'Alcohol': "u'full_bar'", 'RestaurantsDelivery': 'False', 'RestaurantsReservations': 'True', 'BusinessAcceptsCreditCards': 'True', 'BikeParking': 'True', 'HasTV': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'Restaurants

In [8]:
# filter out all review with less than a usefulness of 3
keys_to_delete = []
for i in merged_dataset:
	if merged_dataset[i][0]['useful'] < 3:
		# remove the review
		keys_to_delete.append(i)
garage = []
keys_to_delete = []
for i in merged_dataset:
	try:
		tmp_string = merged_dataset[i][1]['attributes']['BusinessParking']
		# Find and replace False with "False" in tmp_string
		tmp_string = tmp_string.replace("False", "'False'")
		# Do the same for True
		tmp_string = tmp_string.replace("True", "'True'")
		tmp_string = tmp_string.replace("'", '"')
		tmp = json.loads(tmp_string)
		garage.append(tmp['garage'])
	# print(merged_dataset[i][1]['attributes']['BusinessParking']['Garage'])
	except:
		keys_to_delete.append(i)
	# 	pass
for i in keys_to_delete:
	merged_dataset.pop(i)

In [9]:
print(len(merged_dataset))
print(len(garage))

2018
2018


In [10]:
# # Create list of reviews 
# reviews_text_raw = []
# for i in merged_dataset:
# 	reviews_text_raw.append(merged_dataset[i][0]['text'])

In [11]:
review_keys = list(merged_dataset.keys())

In [12]:
merged_dataset[review_keys[0]][0]['text']

'Great food and service! Again. 4 out of the last 5 weeks great every time.. tuesday and Wednesday get right in.  Always looking to make sure that you have a great time!'

In [13]:
#remove punctuation
import string
table = str.maketrans('', '', string.punctuation)

# Remove stop words
from nltk.corpus import stopwords
stop = stopwords.words('english')

# Lemmatize all words
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [14]:
# review_text = []
# for i in reviews_text_raw:
# 	review_text.append(' '.join([word for word in i.lower().split() if word not in stop]))
for i in review_keys:
    review_text = merged_dataset[i][0]['text']
    review_text = ' '.join([word for word in review_text.lower().split() if word not in stop]) #remove stopwords
    review_text = review_text.translate(table) #remove punc.
    review_text = ''.join(c for c in review_text if not c.isdigit()) #remove numbers
    review_text = review_text.replace('  ', ' ') #remove double spaces
    review_text = ' '.join([lemmatizer.lemmatize(word) for word in review_text.split()]) #lemminize
    merged_dataset[i][0]['text'] = review_text
    

In [15]:
print(merged_dataset[review_keys[0]][0]['text'])

great food service again last week great every time tuesday wednesday get right in always looking make sure great time


In [16]:
# # Split sentences into lists
# review_text_list = [[i.split()] for i in review_text]

In [17]:
import gensim.downloader as api
# wv = api.load('word2vec-google-news-300')

In [18]:
wv = api.load('word2vec-google-news-300')

In [19]:
review_text.split()

['charming',
 'bakery',
 'conveniently',
 'location',
 'rt',
 'west',
 'jefferson',
 'best',
 'sugar',
 'cooky',
 'beautifully',
 'packaged',
 'yummy',
 'buy',
 'extra',
 'give',
 'gift',
 'cake',
 'amazing',
 'too']

In [37]:
for i in review_keys:
    single_review_vector = []
    review_text = merged_dataset[i][0]['text']
    for word in review_text.split():
        try:
            single_review_vector.append(np.asarray(wv[word]).astype('float32'))
#             single_review_vector.append(list(wv[word]))
        except:
            print(f'Word {word} not in vocabulary')
            #we are ignoring words that are just combined by accident: fix later???
#     merged_dataset[i][0]['vec_review'] = np.asarray(single_review_vector).astype('float32')
    merged_dataset[i][0]['vec_review'] = single_review_vector
#     merged_dataset[i][0]['vec_review'] = np.array(single_review_vector, dtype=np.float)

Word and not in vocabulary
Word downbest not in vocabulary
Word burren not in vocabulary
Word offkey not in vocabulary
Word livelybutnotraucous not in vocabulary
Word eggmeatmuffin not in vocabulary
Word to not in vocabulary
Word itit not in vocabulary
Word nime not in vocabulary
Word soupbase not in vocabulary
Word chamblee not in vocabulary
Word knute not in vocabulary
Word knute not in vocabulary
Word softmedium not in vocabulary
Word skyy not in vocabulary
Word to not in vocabulary
Word slowy not in vocabulary
Word mcslowerson not in vocabulary
Word fagiole not in vocabulary
Word fusili not in vocabulary
Word gooeyyummydelicious not in vocabulary
Word fungry not in vocabulary
Word fagiole not in vocabulary
Word daylon not in vocabulary
Word osso not in vocabulary
Word bocco not in vocabulary
Word bomer not in vocabulary
Word tsos not in vocabulary
Word pizzapalooza not in vocabulary
Word inwow not in vocabulary
Word and not in vocabulary
Word wasthick not in vocabulary
Word pdx not

Word eyecontact not in vocabulary
Word borracho not in vocabulary
Word pizzayes not in vocabulary
Word foodcleanhuge not in vocabulary
Word cutsdefinitely not in vocabulary
Word a not in vocabulary
Word and not in vocabulary
Word ceasar not in vocabulary
Word aint not in vocabulary
Word notsorry not in vocabulary
Word nutmeggers not in vocabulary
Word minidrink not in vocabulary
Word didnt not in vocabulary
Word to not in vocabulary
Word faneuil not in vocabulary
Word curteous not in vocabulary
Word gourmetportion not in vocabulary
Word yagger not in vocabulary
Word cinara not in vocabulary
Word espelette not in vocabulary
Word noiseincreased not in vocabulary
Word nycs not in vocabulary
Word budweiser not in vocabulary
Word budweiser not in vocabulary
Word draught not in vocabulary
Word burlington not in vocabulary
Word marriott not in vocabulary
Word servicedont not in vocabulary
Word to not in vocabulary
Word usforgot not in vocabulary
Word of not in vocabulary
Word and not in vocab

Word marios not in vocabulary
Word barberstyle not in vocabulary
Word franz not in vocabulary
Word whhaaattt not in vocabulary
Word and not in vocabulary
Word zazas not in vocabulary
Word pastelito not in vocabulary
Word guallava not in vocabulary
Word zazas not in vocabulary
Word trafficwise not in vocabulary
Word sheps not in vocabulary
Word lespalier not in vocabulary
Word michelin not in vocabulary
Word of not in vocabulary
Word betterseems not in vocabulary
Word specialfrozen not in vocabulary
Word goodkeeping not in vocabulary
Word osso not in vocabulary
Word earlyweek not in vocabulary
Word lansdowne not in vocabulary
Word menchies not in vocabulary
Word menchies not in vocabulary
Word baos not in vocabulary
Word to not in vocabulary
Word a not in vocabulary
Word glutenfree not in vocabulary
Word dairyfree not in vocabulary
Word offstreet not in vocabulary
Word themnamely not in vocabulary
Word saythere not in vocabulary
Word yummo not in vocabulary
Word cajunsouthern not in voc

Word semirude not in vocabulary
Word publix not in vocabulary
Word krogers not in vocabulary
Word descently not in vocabulary
Word rossini not in vocabulary
Word moroccon not in vocabulary
Word resevations not in vocabulary
Word multuple not in vocabulary
Word menma not in vocabulary
Word tantan not in vocabulary
Word fruitveggie not in vocabulary
Word dryaged not in vocabulary
Word hennesseys not in vocabulary
Word destinguishing not in vocabulary
Word favourite not in vocabulary
Word 楊枝甘露 not in vocabulary
Word taxtip not in vocabulary
Word wellmaintained not in vocabulary
Word fellini not in vocabulary
Word isabelle not in vocabulary
Word enat not in vocabulary
Word deliverycom not in vocabulary
Word waitedand not in vocabulary
Word callingno not in vocabulary
Word zeus not in vocabulary
Word to not in vocabulary
Word sportsbar not in vocabulary
Word shammel not in vocabulary
Word to not in vocabulary
Word silverwareand not in vocabulary
Word tonkanese not in vocabulary
Word bside n

Word patagonia not in vocabulary
Word kongstyle not in vocabulary
Word flavour not in vocabulary
Word flavour not in vocabulary
Word flavour not in vocabulary
Word chicagostyle not in vocabulary
Word topher not in vocabulary
Word cantonese not in vocabulary
Word appes not in vocabulary
Word nonalco not in vocabulary
Word to not in vocabulary
Word dunwoody not in vocabulary
Word and not in vocabulary
Word pdx not in vocabulary
Word tastyand not in vocabulary
Word allinall not in vocabulary
Word foodtogo not in vocabulary
Word jaclyn not in vocabulary
Word farzad not in vocabulary
Word noriko not in vocabulary
Word noriko not in vocabulary
Word inka not in vocabulary
Word volteada not in vocabulary
Word and not in vocabulary
Word and not in vocabulary
Word to not in vocabulary
Word blt not in vocabulary
Word edgewater not in vocabulary
Word winewater not in vocabulary
Word seeta not in vocabulary
Word porkveggiechicken not in vocabulary
Word ramens not in vocabulary
Word akamaru not in v

Word and not in vocabulary
Word washedup not in vocabulary
Word benihana not in vocabulary
Word benihana not in vocabulary
Word benihana not in vocabulary
Word benihana not in vocabulary
Word a not in vocabulary
Word nonvideo not in vocabulary
Word a not in vocabulary
Word verna not in vocabulary
Word boutonnièrecorsages not in vocabulary
Word texturecrunch not in vocabulary
Word to not in vocabulary
Word diegel not in vocabulary
Word prebirthday not in vocabulary
Word treklike not in vocabulary
Word mechanizedand not in vocabulary
Word blowdrying not in vocabulary
Word flatironing not in vocabulary
Word lubys not in vocabulary
Word soso not in vocabulary
Word disappointintg not in vocabulary
Word papertowel not in vocabulary
Word hinkyness not in vocabulary
Word greatand not in vocabulary
Word awesomelike not in vocabulary
Word minisauna not in vocabulary
Word mattersthe not in vocabulary
Word didnt not in vocabulary
Word fastif not in vocabulary
Word herethank not in vocabulary
Word 

Word runofthemill not in vocabulary
Word runofthemill not in vocabulary
Word placetheyve not in vocabulary
Word methough not in vocabulary
Word cutcolored not in vocabulary
Word devildriver not in vocabulary
Word baretto not in vocabulary
Word baretto not in vocabulary
Word artsyfartsy not in vocabulary
Word mealstandard not in vocabulary
Word garlicy not in vocabulary
Word hushpuppielike not in vocabulary
Word publix not in vocabulary
Word zatarains not in vocabulary
Word mellanzana not in vocabulary
Word pesarattu not in vocabulary
Word pesarattu not in vocabulary
Word pesarattu not in vocabulary
Word brussel not in vocabulary
Word businessthat not in vocabulary
Word geargeek not in vocabulary
Word mcintosh not in vocabulary
Word isnt not in vocabulary
Word outoftowner not in vocabulary
Word evwrywhere not in vocabulary
Word mcgrath not in vocabulary
Word mcgrath not in vocabulary
Word chemabrasions not in vocabulary
Word onlyalas not in vocabulary
Word fattyewwwww not in vocabulary


Word nordstrom not in vocabulary
Word jacketblazer not in vocabulary
Word rollingwood not in vocabulary
Word nordstrom not in vocabulary
Word roxy not in vocabulary
Word doover not in vocabulary
Word carnavon not in vocabulary
Word and not in vocabulary
Word and not in vocabulary
Word a not in vocabulary
Word slowroast not in vocabulary
Word villagio not in vocabulary
Word carbonata not in vocabulary
Word vaq not in vocabulary
Word blt not in vocabulary
Word blt not in vocabulary
Word cheesethey not in vocabulary
Word mentos not in vocabulary
Word halfmade not in vocabulary
Word natasha not in vocabulary
Word embilta not in vocabulary
Word and not in vocabulary
Word sickeningthis not in vocabulary
Word inafter not in vocabulary
Word a not in vocabulary
Word paav not in vocabulary
Word bhature not in vocabulary
Word paav not in vocabulary
Word bhaaji not in vocabulary
Word paav not in vocabulary
Word bhaaji not in vocabulary
Word prethink not in vocabulary
Word burmaby not in vocabulary

Word veras not in vocabulary
Word luigis not in vocabulary
Word harlow not in vocabulary
Word minoux not in vocabulary
Word checkoutsalways not in vocabulary
Word fridaysaturday not in vocabulary
Word dudeman not in vocabulary
Word washingtonstate not in vocabulary
Word nightt not in vocabulary
Word burgerssalads not in vocabulary
Word neighbourhood not in vocabulary
Word dogsive not in vocabulary
Word kez not in vocabulary
Word beckie not in vocabulary
Word hovan not in vocabulary
Word shrwarma not in vocabulary
Word dolomes not in vocabulary
Word innovatiive not in vocabulary
Word garlicy not in vocabulary
Word soso not in vocabulary
Word sessionable not in vocabulary
Word bevmo not in vocabulary


In [38]:
review_df = pd.DataFrame()

In [39]:
for i in review_keys:
    review_df = review_df.append(pd.Series(merged_dataset[i][0]['vec_review']), ignore_index=True)

In [40]:
for col in review_df.columns:
    indcies = [ i for i in range(0,len(review_df)) if type(review_df.loc[i, col]) == float]
    for i in indcies:
#         review_df.at[i, col] = np.asarray(np.zeros(300)).astype('float32')
        review_df.at[i, col] = np.array(list(np.zeros(300)), dtype=np.float)
#         review_df.at[i, col] = list(np.zeros(300))


In [41]:
review_df.head()

,0,1,2,3,4,5,6,7,8,9,...,443,444,445,446,447,448,449,450,451,452
0,"[0.071777344, 0.20800781, -0.028442383, 0.1787...","[-0.18164062, 0.16503906, -0.16601562, 0.35742...","[0.067871094, -0.041259766, 0.100097656, 0.058...","[0.08691406, 0.15136719, -0.02319336, 0.229492...","[0.14257812, 0.072265625, 0.030273438, -0.0383...","[-0.022583008, 0.18457031, -0.10839844, -0.059...","[0.071777344, 0.20800781, -0.028442383, 0.1787...","[0.048339844, -0.19042969, 0.041503906, 0.1215...","[-0.04736328, 0.1875, 0.0022583008, 0.17382812...","[-0.22167969, 0.05810547, 0.17089844, 0.074218...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"[0.10839844, 0.100097656, -0.041992188, 0.1630...","[-0.020874023, 0.021850586, -0.083496094, -0.0...","[-0.14648438, 0.11816406, 0.2421875, -0.076660...","[0.06201172, 0.07519531, 0.10595703, 0.0566406...","[-0.075683594, 0.033691406, -0.064941406, 0.13...","[0.026245117, 0.1328125, -0.0234375, 0.0913085...","[0.043945312, 0.17675781, 0.09472656, -0.08349...","[-0.08251953, 0.022460938, -0.14941406, 0.0991...","[-0.08642578, -0.09667969, -0.08935547, -0.115...","[-0.14355469, -0.1796875, -0.31835938, 0.48437...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[0.09326172, -0.04638672, -0.13378906, 0.04809...","[-0.0005187988, 0.16015625, 0.0016098022, 0.02...","[-0.21582031, 0.014343262, -0.07910156, 0.1933...","[0.16601562, 0.22851562, -0.00390625, 0.175781...","[-0.15136719, -0.033935547, -0.084472656, 0.20...","[-0.16113281, 0.43359375, -0.100097656, 0.0849...","[-0.0005187988, 0.16015625, 0.0016098022, 0.02...","[-0.21582031, 0.014343262, -0.07910156, 0.1933...","[-0.28515625, 0.076171875, 0.041259766, 0.5195...","[-0.096191406, -0.04321289, -0.03515625, 0.143...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"[-0.16015625, -0.0005874634, -0.18261719, 0.10...","[-0.041503906, -0.13183594, -0.19824219, 0.013...","[0.040527344, 0.0625, -0.017456055, 0.07861328...","[-0.18164062, 0.16503906, -0.16601562, 0.35742...","[0.040527344, 0.0625, -0.017456055, 0.07861328...","[-0.087402344, -0.13378906, -0.044921875, 0.37...","[-0.16113281, 0.43359375, -0.100097656, 0.0849...","[-0.20605469, -0.29492188, 0.12597656, 0.29296...","[-0.17382812, 0.024047852, -0.030151367, 0.277...","[0.20605469, 0.22753906, -0.21386719, 0.064941...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0,

In [42]:
# Convert garage from string False or True to 0 or 1
garage = [0  if i == "False" else 1 for i in garage]

In [43]:
garage = np.array(garage)

In [68]:
# review_numpy = np.array(review_df)

In [92]:
# Split the review vector and garage tensors into training and testing sets
from sklearn.model_selection import train_test_split
review_vector_train, review_vector_test, garage_train, garage_test = train_test_split(review_df, garage, test_size=0.2)

In [84]:
import tensorflow as tf

In [112]:
garage_train = tf.convert_to_tensor(garage_train, dtype=tf.float32)
garage_test = tf.convert_to_tensor(garage_test, dtype=tf.float32)

In [113]:
review_vector_train_tens = tf.constant(([review_vector_train[i] for i in review_vector_train.columns]))
review_vector_test_tens = tf.constant(([review_vector_test[i] for i in review_vector_test.columns]))

In [114]:
# Train lstm model
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping

# Initialize the model
model = Sequential()
# Add an LSTM layer with 100 units
model.add(LSTM(100, input_shape=(review_vector_train_tens.shape[1], review_vector_train_tens.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
# Add an LSTM layer with 100 units
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
# Add an LSTM layer with 100 units
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
# Add an LSTM layer with 100 units
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
# Softmax layer
model.add(Dense(1, activation='sigmoid'))

# Train the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
model.fit(review_vector_train_tens, garage_train, epochs=1, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

12/12 [==============================] - 202s 17s/step - loss: 0.2824 - accuracy: 0.9515 - val_loss: 0.0032 - val_accuracy: 1.0000
